In [1]:
%matplotlib widget

import numpy as np
import matplotlib.pyplot as plt

import xpart as xp
import xtrack as xt

fname_model = './generate_models/lhc_at_injection_obstacle.json'

collider = xt.Multiline.from_json(fname_model)
collider.build_trackers()

nemitt_x = 3.5e-6
nemitt_y = 3.5e-6
num_particles = 1000

Done loading line from dict.           
Done loading line from dict.           
Done loading line from dict.           
Done loading line from dict.           
Compiling ContextCpu kernels...
Done compiling ContextCpu kernels.
Compiling ContextCpu kernels...
Done compiling ContextCpu kernels.
Compiling ContextCpu kernels...
Done compiling ContextCpu kernels.
Compiling ContextCpu kernels...
Done compiling ContextCpu kernels.


## Investigating an obstacle

We start by twissing. We need to build a bunch of particles. Since we will reuse the bunch later, we copy it as a template.

In [2]:
bunch = xp.generate_matched_gaussian_bunch(
    line=collider.lhcb1,
    sigma_z=10e-2,
    nemitt_x=nemitt_x,
    nemitt_y=nemitt_y,
    num_particles=num_particles,
)

# We keep a copy of the genereated bunch (we'll used it later)
bunch0 = bunch.copy()

*** Maximum RMS bunch length 0.11789515101464537m.
... distance to target bunch length: -1.0000e-01
... distance to target bunch length: 1.4638e-02
... distance to target bunch length: 1.3573e-02
... distance to target bunch length: -1.0721e-03
... distance to target bunch length: 1.6142e-03
... distance to target bunch length: 7.1106e-05
... distance to target bunch length: 2.9063e-06
... distance to target bunch length: -4.0654e-10
... distance to target bunch length: 1.9477e-07
--> Bunch length: 0.09999999959345827
--> Emittance: 0.6422072248004196
Warning! scale_with_transverse_norm_emitt is deprecated. Use nemitt_x and nemitt_y instead.


### Track the bunch

We track the bunch and observe the losses:

In [4]:
collider.lhcb1.track(bunch, num_turns=5)

We check how many particles are lost

In [5]:
print(f'Lost beam fraction: {np.sum(bunch.state <= 0)/num_particles*100:.2f}%')

Lost beam fraction: 35.10%


To find out where we lose the particles, we filter them, and inspect the element where the losses occur:

In [6]:
lost_particles = bunch.filter(bunch.state <= 0)

# Find the name of the element where we lose most particles
from collections import Counter
i_most_losses = Counter(lost_particles.at_element).most_common(1)[0][0]

print(f'The element where we lose most particles is called `{collider.lhcb1.element_names[i_most_losses]}`!!!!')

The element where we lose most particles is called `obstacle`!!!!


### Plot the lost particles at the obstacle

The element where we lose the particles is of an aperture type:

In [8]:
collider.lhcb1.elements[i_most_losses]

We can plot the lost particles and the obstacle cross-section:

In [9]:
# Plot the lost particles and the obstacle
obstacle = collider.lhcb1['obstacle']
fig, ax = plt.subplots()
ax.plot(lost_particles.x, lost_particles.y, '.', markersize=1)
ax.plot(obstacle.x_vertices, obstacle.y_vertices, 'k')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Correct the orbit to avoid the obstacle

Find the accelerator element closest to the obstacle:

In [11]:
tw = collider.lhcb1.twiss()
s_obstacle = tw['s', 'obstacle']

tw.rows[s_obstacle-5.:s_obstacle+5.:'s']

TwissTable: 8 rows, 47 cols
name                               s           x           px            y ...
mb.c15r8.b1..1               17296.5 1.68121e-06  -2.1669e-08 -2.04115e-07
drift_7471                   17296.5 1.68121e-06 -1.82169e-08 -2.04115e-07
obstacle                     17301.2 1.59437e-06 -1.82169e-08 -2.19192e-07
chamber                      17301.2 1.59437e-06 -1.82169e-08 -2.19192e-07
mb.c15r8.b1                  17301.2 1.59437e-06 -1.82169e-08 -2.19192e-07
drift_7472                   17301.2 1.59437e-06 -1.82169e-08 -2.19192e-07
mb.c15r8.b1..2                 17306 1.50754e-06 -1.82169e-08  -2.3427e-07
drift_7473                     17306 1.50754e-06 -1.47646e-08  -2.3427e-07

In [12]:
element_close_to_obstacle = 'mb.c15r8.b1'

Find the vertical correctors close to the obstacle:

In [13]:
# Correctors upstream
tw.rows[s_obstacle-300.:s_obstacle:'s', 'mcbv.*']

TwissTable: 2 rows, 47 cols
name                               s           x          px            y ...
mcbv.11r8.b1                   17101 1.63309e-06  1.1516e-08  2.39107e-07
mcbv.13r8.b1                 17207.9 1.32796e-06 1.71837e-08 -4.27231e-08

In [14]:
# Correctors downstream
tw.rows[s_obstacle:s_obstacle+300.:'s', 'mcbv.*']

TwissTable: 3 rows, 47 cols
name                               s           x          px            y ...
mcbv.15r8.b1                 17314.8   1.474e-06 2.39467e-08 -2.45224e-07
mcbv.17r8.b1                 17421.7 1.62308e-06 2.18696e-08   4.2666e-08
mcbv.19r8.b1                 17528.6 1.47708e-06  1.5106e-08  2.45234e-07

Find out the knob that controls the corrector:

In [15]:
v_corr = 'mcbv.15r8.b1' # We check only one (the others follow the same naming convention)
collider.lhcb1.element_refs[v_corr].ksl[0]._expr

vars['acbv15.r8b1']

Perform a match to move the beam vertically at the obstacle:

In [16]:
collider.lhcb1_co_ref.match( # We use the reference line where there is no obstacle
    method='4d',
    ele_start='bpm.10r8.b1',
    ele_stop='bpm.18r8.b1',
    twiss_init=tw.get_twiss_init(at_element='bpm.10r8.b1'),
    vary=[
        xt.Vary(name='acbv11.r8b1', step=1e-10),
        xt.Vary(name='acbv13.r8b1', step=1e-10),
        xt.Vary(name='acbv15.r8b1', step=1e-10),
        xt.Vary(name='acbv17.r8b1', step=1e-10),
    ],
    targets=[
        # I want the vertical orbit to be at 10 mm from obstacle with flat angle
        xt.Target('y', at=element_close_to_obstacle, value=10e-3, tol=1e-4, scale=1),
        xt.Target('py', at=element_close_to_obstacle, value=0, tol=1e-6, scale=1000),
        # I want the bump to be closed
        xt.Target('y', at='bpm.18r8.b1', value=tw['y', 'bpm.18r8.b1'],
                  tol=1e-6, scale=1),
        xt.Target('py', at='bpm.18r8.b1', value=tw['py', 'bpm.18r8.b1'],
                   tol=1e-7, scale=1000),
    ]
)

Matching: twiss call n. 14       



{'res': array([1.41071721e-04, 4.94191561e-05, 1.38307965e-04, 5.39739016e-05]),
 'info': {'nfev': 3,
  'njev': 1,
  'fjac': array([[-0.01515282,  0.34369272,  0.02433904, -0.93864441],
         [-0.06472875, -0.9367265 ,  0.04764805, -0.34071001],
         [ 0.9279103 , -0.06508155, -0.36389453, -0.04824548],
         [ 0.36682836,  0.01353392,  0.92990217,  0.0231461 ]]),
  'r': array([-2437.71715349,  1759.18644455,  2222.60880003,  -937.48083101,
         -1909.72972861,   809.7158334 ,  -338.37052968,    92.76109256,
           -65.78300285,    67.96581106]),
  'qtf': array([-4.31559301e-05,  1.94365551e-04,  7.40094962e-06, -5.86443612e-07]),
  'fvec': array([ 0.,  0.,  0., -0.])},
 'ier': 1,
 'mesg': 'The solution converged.'}

Plot the orbit after the correction

In [17]:
tw_bump = collider.lhcb1_co_ref.twiss(method='4d')
fig, ay = plt.subplots()

ay.plot(tw.s, tw.y*1000, label='y (before)')
ay.plot(tw_bump.s, tw_bump.y*1000, label='y (after)')

# Obstacle
ay.axvline(x=collider.lhcb1.get_s_position('obstacle'), color='r', linestyle='-', alpha=0.5, label='obstacle')

# Vertical correctors
ay.axvline(x=collider.lhcb1.get_s_position('mcbv.11r8.b1'), color='k', linestyle='--', alpha=0.5, label='mcbv.11r8.b1')
ay.axvline(x=collider.lhcb1.get_s_position('mcbv.13r8.b1'), color='k', linestyle='--', alpha=0.5, label='mcbv.13r8.b1')
ay.axvline(x=collider.lhcb1.get_s_position('mcbv.15r8.b1'), color='k', linestyle='--', alpha=0.5, label='mcbv.15r8.b1')
ay.axvline(x=collider.lhcb1.get_s_position('mcbv.17r8.b1'), color='k', linestyle='--', alpha=0.5, label='mcbv.17r8.b1')

# Vertical match limit elements
ay.axvline(x=collider.lhcb1.get_s_position('bpm.10r8.b1'), color='g', linestyle='--', alpha=0.5, label='bpm.10r8.b1')
ay.axvline(x=collider.lhcb1.get_s_position('bpm.18r8.b1'), color='g', linestyle='--', alpha=0.5, label='bpm.18r8.b1')

#ay.axhline(y=18., color='b', linestyle='-', alpha=0.5, label='aperture limit')

ay.set_xlim(collider.lhcb1.get_s_position('bpm.10r8.b1') - 10,
            collider.lhcb1.get_s_position('bpm.18r8.b1') + 10)


ay.set_xlabel('s [m]')
ay.set_ylabel('y [mm]')
ay.set_ylim(-0.5, 25)
fig.tight_layout()
fig.subplots_adjust(right=0.75)
fig.legend(loc='right')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

We track the particles again

In [18]:
bunch = bunch0.copy()
collider.lhcb1.track(bunch, num_turns=5)
print(f'Lost beam fraction: {np.sum(bunch.state <= 0)/num_particles*100:.2f}%')

Lost beam fraction: 100.00%


We lost the whole bunch!!!! What is happening?

Let's add in the plot the vertical beam pipe size

In [34]:

fig, ay = plt.subplots()

ay.plot(tw.s, tw.y*1000, label='y (before)')
ay.plot(tw_bump.s, tw_bump.y*1000, label='y (after)')

# Obstacle
ay.axvline(x=collider.lhcb1.get_s_position('obstacle'), color='r', linestyle='-', alpha=0.5, label='obstacle')

# Vertical correctors
ay.axvline(x=collider.lhcb1.get_s_position('mcbv.11r8.b1'), color='k', linestyle='--', alpha=0.5, label='mcbv.11r8.b1')
ay.axvline(x=collider.lhcb1.get_s_position('mcbv.13r8.b1'), color='k', linestyle='--', alpha=0.5, label='mcbv.13r8.b1')
ay.axvline(x=collider.lhcb1.get_s_position('mcbv.15r8.b1'), color='k', linestyle='--', alpha=0.5, label='mcbv.15r8.b1')
ay.axvline(x=collider.lhcb1.get_s_position('mcbv.17r8.b1'), color='k', linestyle='--', alpha=0.5, label='mcbv.17r8.b1')

# Vertical match limit elements
ay.axvline(x=collider.lhcb1.get_s_position('bpm.10r8.b1'), color='g', linestyle='--', alpha=0.5, label='bpm.10r8.b1')
ay.axvline(x=collider.lhcb1.get_s_position('bpm.18r8.b1'), color='g', linestyle='--', alpha=0.5, label='bpm.18r8.b1')

ay.axhline(y=18., color='b', linestyle='-', alpha=0.5, label='aperture limit')

ay.set_xlim(collider.lhcb1.get_s_position('bpm.10r8.b1') - 10,
            collider.lhcb1.get_s_position('bpm.18r8.b1') + 10)


ay.set_xlabel('s [m]')
ay.set_ylabel('y [mm]')
ay.set_ylim(-0.5, 25)
fig.tight_layout()
fig.subplots_adjust(right=0.75)
fig.legend(loc='right')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

We manage to pass the obstacle but we are bent far off by the defocusing quadrupole

In [19]:
# Less aggressive bump
collider.lhcb1_co_ref.match(
    method='4d',
    ele_start='bpm.10r8.b1',
    ele_stop='bpm.18r8.b1',
    twiss_init=tw.get_twiss_init(at_element='bpm.10r8.b1'),
    vary=[
        xt.Vary(name='acbv11.r8b1', step=1e-10),
        xt.Vary(name='acbv13.r8b1', step=1e-10),
        xt.Vary(name='acbv15.r8b1', step=1e-10),
        xt.Vary(name='acbv17.r8b1', step=1e-10),
    ],
    targets=[
        # I want the vertical orbit to be at 1.5 mm from obstacle with flat angle
        xt.Target('y', at=element_close_to_obstacle, value=1.5e-3, tol=1e-4, scale=1),
        xt.Target('py', at=element_close_to_obstacle, value=0, tol=1e-6, scale=1000),
        # I want the bump to be closed
        xt.Target('y', at='bpm.18r8.b1', value=tw['y', 'bpm.18r8.b1'],
                  tol=1e-6, scale=1),
        xt.Target('py', at='bpm.18r8.b1', value=tw['py', 'bpm.18r8.b1'],
                   tol=1e-7, scale=1000),
    ]
)

Matching: twiss call n. 14       



{'res': array([2.11665918e-05, 7.40394440e-06, 2.07571017e-05, 8.09603781e-06]),
 'info': {'nfev': 3,
  'njev': 1,
  'fjac': array([[-0.01512451,  0.3444248 ,  0.02430979, -0.93837724],
         [-0.06479773, -0.93645396,  0.04769374, -0.34143892],
         [ 0.92790144, -0.06513277, -0.36390212, -0.04828952],
         [ 0.36683977,  0.01353713,  0.92989762,  0.0231462 ]]),
  'r': array([-2438.50452874,  1762.44255705,  2222.08152909,  -937.18815497,
         -1907.82717614,   809.60368005,  -339.21864413,    92.78877315,
           -65.83203107,    67.96605881]),
  'qtf': array([-6.68232883e-05,  3.01626059e-04,  1.15169544e-05, -9.12747466e-07]),
  'fvec': array([-0., -0., -0.,  0.])},
 'ier': 1,
 'mesg': 'The solution converged.'}

In [20]:
# Let's try to track again
bunch = bunch0.copy()
collider.lhcb1.track(bunch, num_turns=5)
print(f'Lost beam fraction: {np.sum(bunch.state <= 0)/num_particles*100:.2f}%')

Lost beam fraction: 1.00%


There are still some losses. We can plot the beam at the obstacle, to see what happens. To do that, we need to insert a monitor at the obstacle:

In [21]:
collider.lhcb1.discard_tracker()
monitor_obstacle = xt.ParticlesMonitor(start_at_turn=0, stop_at_turn=4,
                                       num_particles=num_particles)
collider.lhcb1.insert_element(
    index='obstacle',
    element=monitor_obstacle,
    name='monitor_obstacle',
)


In [22]:
collider.lhcb1.build_tracker()
bunch = bunch0.copy()
collider.lhcb1.track(bunch, num_turns=5)

Compiling ContextCpu kernels...
Done compiling ContextCpu kernels.


In [23]:
obstacle = collider.lhcb1['obstacle']
fig, ax = plt.subplots()
ax.plot(monitor_obstacle.x, monitor_obstacle.y, '.', markersize=1)
ax.plot(obstacle.x_vertices, obstacle.y_vertices, 'k')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Some particles are still lost at the obstacle. We can shift the beam a little horizontaly to fix this issue completely. As before, we find the knob that controls the horizontal corrector close to the beam.

In [24]:
tw.rows[s_obstacle-300.:s_obstacle+300.:'s', 'mcbh.*|obstacle']

TwissTable: 6 rows, 47 cols
name                               s           x           px            y ...
mcbh.12r8.b1                 17154.4 2.68505e-06  -3.8315e-08  5.64296e-08
mcbh.14r8.b1                 17261.3 2.66895e-06 -3.54758e-08 -9.29942e-08
obstacle                     17301.2 1.59437e-06 -1.82169e-08 -2.19192e-07
mcbh.16r8.b1                 17368.2 3.14394e-06  -4.1771e-08  -5.6977e-08
mcbh.18r8.b1                 17475.1 3.17905e-06 -4.49976e-08   9.2981e-08
mcbh.20r8.b1                   17582 2.70406e-06  -3.8703e-08  5.69987e-08

In [25]:
h_corr = 'mcbh.14r8.b1'
collider.lhcb1.element_refs[h_corr].knl[0]._expr

(-vars['acbh14.r8b1'])

Match again:

In [26]:
collider.lhcb1_co_ref.match(
    method='4d',
    ele_start='bpm.10r8.b1',
    ele_stop='bpm.20r8.b1',
    twiss_init=tw.get_twiss_init(at_element='bpm.10r8.b1'),
    vary=[
        xt.Vary(name='acbh12.r8b1', step=1e-10),
        xt.Vary(name='acbh14.r8b1', step=1e-10),
        xt.Vary(name='acbh16.r8b1', step=1e-10),
        xt.Vary(name='acbh18.r8b1', step=1e-10),
    ],
    targets=[
        # I want the vertical orbit to be at 1 mm from obstacle with flat angle
        xt.Target('x', at=element_close_to_obstacle, value=-3e-3, tol=1e-4, scale=1),
        xt.Target('px', at=element_close_to_obstacle, value=0, tol=1e-6, scale=1000),
        # I want the bump to be closed
        xt.Target('x', at='bpm.20r8.b1', value=tw['x', 'bpm.20r8.b1'],
                  tol=1e-6, scale=1),
        xt.Target('px', at='bpm.20r8.b1', value=tw['px', 'bpm.20r8.b1'],
                   tol=1e-7, scale=1000),
    ]
)

Matching: twiss call n. 14       



{'res': array([-1.73924801e-05, -4.09524495e-05, -1.68483461e-05, -4.10394690e-05]),
 'info': {'nfev': 3,
  'njev': 1,
  'fjac': array([[-0.05221423,  0.92844264,  0.00177971, -0.3677836 ],
         [ 0.05938394, -0.36297189, -0.09363506, -0.92517967],
         [ 0.36154947,  0.0592123 , -0.9257441 ,  0.09366822],
         [ 0.92899365,  0.05234099,  0.36637025,  0.00201463]]),
  'r': array([-2540.1688211 ,  1812.0392203 ,   343.38147178,  -876.02017194,
          1875.62537899,   863.05945387, -2224.21326179,   -92.16310409,
            64.28227243,    67.93953527]),
  'qtf': array([-3.80194273e-04, -5.97367310e-04,  4.36203199e-05, -1.73197346e-06]),
  'fvec': array([-0., -0., -0., -0.])},
 'ier': 1,
 'mesg': 'The solution converged.'}

Track and inspect the beam again, this time there should be no losses:

In [28]:
bunch = bunch0.copy()
collider.lhcb1.track(bunch, num_turns=5)

# See the beam at the obstacle:
obstacle = collider.lhcb1['obstacle']
fig, ax = plt.subplots()
ax.plot(monitor_obstacle.x, monitor_obstacle.y, '.', markersize=1)
ax.plot(obstacle.x_vertices, obstacle.y_vertices, 'k')

# We don't lose so many particles anymore
print(f'Lost beam fraction: {np.sum(bunch.state <= 0)/num_particles*100:.2f}%')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Lost beam fraction: 0.00%


In [29]:
tw_corrected = collider.lhcb1.twiss()

In [30]:
fig, (ax, ay) = plt.subplots(nrows=2, ncols=1, sharex=True)

# ==X==
ax.plot(tw.s, tw.x*1000, label='x (before)')
ax.plot(tw_corrected.s, tw_corrected.x*1000, label='x (after)')

# Obstacle
ax.axvline(x=collider.lhcb1.get_s_position('obstacle'), color='r', linestyle='-', alpha=0.5, label='obstacle')

# Horizontal correctors
ax.axvline(x=collider.lhcb1.get_s_position('mcbh.12r8.b1'), color='k', linestyle='--', alpha=0.5, label='mcbh.12r8.b1')
ax.axvline(x=collider.lhcb1.get_s_position('mcbh.14r8.b1'), color='k', linestyle='--', alpha=0.5, label='mcbh.14r8.b1')
ax.axvline(x=collider.lhcb1.get_s_position('mcbh.16r8.b1'), color='k', linestyle='--', alpha=0.5, label='mcbh.16r8.b1')
ax.axvline(x=collider.lhcb1.get_s_position('mcbh.18r8.b1'), color='k', linestyle='--', alpha=0.5, label='mcbh.18r8.b1')

# Horizontal match limit elements
ax.axvline(x=collider.lhcb1.get_s_position('bpm.10r8.b1'), color='g', linestyle='--', alpha=0.5, label='bpm.10r8.b1')
ax.axvline(x=collider.lhcb1.get_s_position('bpm.20r8.b1'), color='g', linestyle='--', alpha=0.5, label='bpm.20r8.b1')

# Aperture limit
ax.axhline(y=22., color='b', linestyle='-', alpha=0.5)
ax.axhline(y=-22., color='b', linestyle='-', alpha=0.5)

ax.set_ylabel('x [mm]')
ax.set_ylim(-10, 0.5)
ay.set_xlim(collider.lhcb1.get_s_position('bpm.10r8.b1') - 10,
            collider.lhcb1.get_s_position('bpm.20r8.b1') + 10)
ax.legend(loc='right', bbox_to_anchor=(1.4, 0.5))


# ==Y==
ay.plot(tw.s, tw.y*1000, label='y (before)')
ay.plot(tw_corrected.s, tw_corrected.y*1000, label='y (after)')

# Obstacle
ay.axvline(x=collider.lhcb1.get_s_position('obstacle'), color='r', linestyle='-', alpha=0.5, label='obstacle')

# Vertical correctors
ay.axvline(x=collider.lhcb1.get_s_position('mcbv.11r8.b1'), color='k', linestyle='--', alpha=0.5, label='mcbv.11r8.b1')
ay.axvline(x=collider.lhcb1.get_s_position('mcbv.13r8.b1'), color='k', linestyle='--', alpha=0.5, label='mcbv.13r8.b1')
ay.axvline(x=collider.lhcb1.get_s_position('mcbv.15r8.b1'), color='k', linestyle='--', alpha=0.5, label='mcbv.15r8.b1')
ay.axvline(x=collider.lhcb1.get_s_position('mcbv.17r8.b1'), color='k', linestyle='--', alpha=0.5, label='mcbv.17r8.b1')

# Vertical match limit elements
ay.axvline(x=collider.lhcb1.get_s_position('bpm.10r8.b1'), color='g', linestyle='--', alpha=0.5, label='bpm.10r8.b1')
ay.axvline(x=collider.lhcb1.get_s_position('bpm.18r8.b1'), color='g', linestyle='--', alpha=0.5, label='bpm.18r8.b1')

# Aperture limit
ay.axhline(y=18., color='b', linestyle='-', alpha=0.5)
ay.axhline(y=-18., color='b', linestyle='-', alpha=0.5)
ay.legend(loc='right', bbox_to_anchor=(1.4, 0.5))


ay.set_xlabel('s [m]')
ay.set_ylabel('y [mm]')
ay.set_ylim(-0.5, 10)
fig.subplots_adjust(right=0.75)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
np.trapz(tw.betx, tw.s)/tw.circumference